# Progetto Foundations of Computer Science
### Giulia Mura, matricola: 860910

In [1]:
#importo librerie necessarie
import pandas as pd
import numpy as np
import datetime
from pandas.core.dtypes.missing import isna, notna

In [2]:
#importo files
country_stats_file = '/Users/giulia/FCS/additional-kiva-snapshot/country_stats.csv'
lenders_file = '/Users/giulia/FCS/additional-kiva-snapshot/lenders.csv'
loans_lenders_file = '/Users/giulia/FCS/additional-kiva-snapshot/loans_lenders.csv'
loans_file = '/Users/giulia/FCS/additional-kiva-snapshot/loans.csv'

### 1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [3]:
#come primo step, leggo il file csv ancora da normalizzare e mostro le prime righe per visualizzare il problema
lenders_to_normalize = pd.read_csv(loans_lenders_file)
lenders_to_normalize.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [4]:
#utilizzo la funzione explode e attraverso il metodo split separo la stringa di lenders per ogni loan_id in
#una nuova lista (nel corso dello sviluppo del progetto ho notato che certi lenders presentavano uno spazio bianco
#prima del nome quindi applico str.strip per eliminarli e non avere problemi successivamente)
lenders_normalized = lenders_to_normalize.assign(lenders=lenders_to_normalize['lenders'].str.split(',')).explode('lenders').reset_index(drop=True)
lenders_normalized['lenders'] = lenders_normalized['lenders'].str.strip()

In [5]:
#mostro le prime righe
lenders_normalized.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


### 2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [6]:
#leggo e mostro il file csv loans accedendo direttamente alle due colonne che andrò ad utilizzare per il calcolo
#dei giorni.
loans = pd.read_csv(loans_file, parse_dates=['disburse_time', 'planned_expiration_time'])

In [7]:
#creo la lista date_loans nella quale includo solo le righe in cui l'anno della scadenza corrisponde allo stesso
#dell'esborso o lo segue, in quanto non ha senso logico che la data di scadenza preceda quella di esborso.
date_loans = loans.loc[loans['planned_expiration_time'].dt.year>=loans['disburse_time'].dt.year]

In [8]:
#inserisco in loans una nuova colonna il quale risultato sarà dato dalla sottrazione di planned_expiration_time e 
#disburse_time, visualizzo poi come uniche colonne il loan_id e la corrispondente durata del prestito.
date_loans['duration']=date_loans['planned_expiration_time'] - date_loans['disburse_time']
date_loans[['loan_id', 'disburse_time', 'planned_expiration_time', 'duration']].head()

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,loan_id,disburse_time,planned_expiration_time,duration
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,53 days 19:30:06
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,96 days 14:25:07
2,658010,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,37 days 13:10:05
3,659347,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,34 days 19:10:02
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,57 days 22:10:02


### 3. Find the lenders that have funded at least twice.

In [9]:
#non avendo nel file loans una colonna dedicata ai lenders, creo una lista in cui collego la colonna "status" e
#il file lenders_normalized.
loans_lenders_status = lenders_normalized.merge(loans[['loan_id', 'status']], on = 'loan_id', how = 'left')
loans_lenders_status.head()

,loan_id,lenders,status
0,483693,muc888,funded
1,483693,sam4326,funded
2,483693,camaran3922,funded
3,483693,lachheb1865,funded
4,483693,rebecca3499,funded


In [10]:
#creo la lista lend_funded dando il comando per visualizzare i soli status "funded" ed infine faccio il conteggio
#della presenza di ogni lenders mostrando solo quelli che risultano più di una volta.
lend_funded = loans_lenders_status[loans_lenders_status.status=='funded']
lend_funded_2more = lend_funded['lenders'].value_counts()
lend_funded_2more[lend_funded_2more>1]

gooddogg1           167512
trolltech4460       141268
gmct                123164
nms                  96713
themissionbeltco     74067
                     ...  
ken5471                  2
james40645191            2
john43273183             2
wilma5538                2
eleri2992                2
Name: lenders, Length: 893981, dtype: int64

### 4. For each country, compute how many loans have involved that country as borrowers.

In [11]:
#utilizzo la funzione size per visualizzare il numero di elementi della colonna oggetto della funzione group by.
loans.groupby(['country_code']).size()

country_code
AF     2337
AL     3075
AM    13952
AZ    10172
BA      608
      ...  
XK     2178
YE     4206
ZA      633
ZM     1277
ZW     5513
Length: 95, dtype: int64

### 5. For each country, compute the overall amount of money borrowed.

In [12]:
#in questo caso faccio la somma dei valori della colonna loan_amount continuando a raggruppare rispetto alla
#colonna country_code.
loans.groupby('country_code')['loan_amount'].sum()

country_code
AF     1967950.0
AL     4307350.0
AM    22950475.0
AZ    14784625.0
BA      477250.0
         ...    
XK     3083025.0
YE     3444000.0
ZA     1006525.0
ZM     1978975.0
ZW     5851875.0
Name: loan_amount, Length: 95, dtype: float64

### 6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [13]:
#calcolo la somma totale della colonna loan_amount, valore il quale mi serve per ripetere il procedimento del punto
#precedente ma il quale sottopongo alla divisione per il tot_amount moltiplicato per 100 così ottenendo appunto
#la percentuale di prestito di ogni country sul totale.
tot_amount = loans['loan_amount'].sum()
loans.groupby('country_code')['loan_amount'].sum()/tot_amount*100

country_code
AF    0.166573
AL    0.364586
AM    1.942589
AZ    1.251410
BA    0.040396
        ...   
XK    0.260955
YE    0.291509
ZA    0.085195
ZM    0.167506
ZW    0.495318
Name: loan_amount, Length: 95, dtype: float64

### 7. Like the three previous points, but split for each year (with respect to disburse time).

In [14]:
#seguo lo stesso procedimento del punto 4 inserendo però la funzione pd.Grouper, la quale mi permette di 
#selezionare tramite key la colonna su cui effettuo la funzione sum, specificando però il paramentro freq='Y'
#così raggruppando la colonna specificata per la frequenza dell'anno.
#utilizzo la funzione size per visualizzare il numero di elementi della colonna oggetto della funzione group by.
loans.groupby(['country_code', pd.Grouper(key='disburse_time', freq='Y')]).size()

country_code  disburse_time            
AF            2007-12-31 00:00:00+00:00     408
              2008-12-31 00:00:00+00:00     370
              2009-12-31 00:00:00+00:00     678
              2010-12-31 00:00:00+00:00     632
              2011-12-31 00:00:00+00:00     247
                                           ... 
ZW            2013-12-31 00:00:00+00:00     426
              2014-12-31 00:00:00+00:00    2078
              2015-12-31 00:00:00+00:00     600
              2016-12-31 00:00:00+00:00     808
              2017-12-31 00:00:00+00:00    1079
Length: 745, dtype: int64

In [15]:
#faccio lo stesso procedimento per cui raggruppo per country_code e utilizzo pd.grouper per dividere ulteriormente
#la classificazione rispetto all'anno di disburse_time, ma eseguo la funzione sum sulla colonna loan_amount.
loans.groupby(['country_code', pd.Grouper(key='disburse_time', freq='Y')])['loan_amount'].sum()

country_code  disburse_time            
AF            2007-12-31 00:00:00+00:00     194975.0
              2008-12-31 00:00:00+00:00     365375.0
              2009-12-31 00:00:00+00:00     585125.0
              2010-12-31 00:00:00+00:00     563350.0
              2011-12-31 00:00:00+00:00     245125.0
                                             ...    
ZW            2013-12-31 00:00:00+00:00     678525.0
              2014-12-31 00:00:00+00:00    1311575.0
              2015-12-31 00:00:00+00:00     723625.0
              2016-12-31 00:00:00+00:00     788600.0
              2017-12-31 00:00:00+00:00    1237600.0
Name: loan_amount, Length: 745, dtype: float64

In [16]:
#eseguo gli stessi passaggi del codice precedente calcolando la percentuale utilizzando il tot_amount_year che
#calcolo subito. 
tot_amount_year = loans['loan_amount'].sum()
loans.groupby(['country_code', pd.Grouper(key='disburse_time', freq='Y')])['loan_amount'].sum()/tot_amount_year*100

country_code  disburse_time            
AF            2007-12-31 00:00:00+00:00    0.016503
              2008-12-31 00:00:00+00:00    0.030926
              2009-12-31 00:00:00+00:00    0.049527
              2010-12-31 00:00:00+00:00    0.047683
              2011-12-31 00:00:00+00:00    0.020748
                                             ...   
ZW            2013-12-31 00:00:00+00:00    0.057432
              2014-12-31 00:00:00+00:00    0.111015
              2015-12-31 00:00:00+00:00    0.061250
              2016-12-31 00:00:00+00:00    0.066749
              2017-12-31 00:00:00+00:00    0.104754
Name: loan_amount, Length: 745, dtype: float64

### 8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [17]:
#attraverso la funzione count applicata su una group by rispetto la colonna loan_id, ottengo quante volte si ripete
#tale loan_id, ovvero quanti lenders vi hanno preso parte nel finanziamento.
number_lenders = lenders_normalized.groupby('loan_id').count().reset_index()
number_lenders.head()

,loan_id,lenders
0,84,3
1,85,2
2,86,3
3,88,3
4,89,4


In [18]:
#unisco la colonna "loan_amount" del file loans al file lenders_normalized in modo da avere in riga con il loan_id
#e i rispettivi lenders, il totale del prestito.
lenders_normalized['loan_amount'] = lenders_normalized['loan_id'].map(loans.set_index('loan_id')['loan_amount'])

In [19]:
#unisco la lista "number_lenders" precedentemente creata, a lenders_normalized. 
lenders_normalized['number_lenders'] = lenders_normalized['loan_id'].map(number_lenders.set_index('loan_id')['lenders'])

In [20]:
#supponendo quindi che, per i prestiti ai quali hanno partecipato più lenders essi abbiano contribuito con lo
#stesso ammontare, inserisco una nuova colonna al dataframe che rappresenti il singolo contributo del lender a quel
#determinato loan_id dividendo il loan_amount per il numero di lenders che vi hanno partecipato.
lenders_normalized['loan_amount_per_lender'] = lenders_normalized['loan_amount']/lenders_normalized['number_lenders']

In [21]:
#mostro il risultato di quanto applicato finora 
lenders_normalized.head()

,loan_id,lenders,loan_amount,number_lenders,loan_amount_per_lender
0,483693,muc888,1225.0,40,30.625
1,483693,sam4326,1225.0,40,30.625
2,483693,camaran3922,1225.0,40,30.625
3,483693,lachheb1865,1225.0,40,30.625
4,483693,rebecca3499,1225.0,40,30.625


In [22]:
#infine, dato che ogni lender può aver finanziato più di un prestito, effettuo una group by sulla colonna lenders 
#e applico la funzione sum sulla colonna loan_amount_per_lender in modo tale da visualizzare l'importo totale
#che ogni lender ha sborsato. 
lenders_lent = lenders_normalized.groupby('lenders')['loan_amount_per_lender'].sum().reset_index()
lenders_lent.head()

,lenders,loan_amount_per_lender
0,000,1764.285078
1,00000,1380.693644
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117


### 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [23]:
#il dataframe presenta molti valori nulli nella colonna di nostro interesse, ovvero country_code.
country_stats = pd.read_csv(country_stats_file)
lenders = pd.read_csv(lenders_file)
lenders = lenders.rename(columns = {'permanent_name': 'lenders'})
lenders.head()

,lenders,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


In [24]:
#creo una lista in cui riporto il numero di volte in cui si presenta un dato country_code nel dataframe lenders
freq_country = lenders['country_code'].value_counts().reset_index().rename(columns = {'index': 'country_code', 'country_code': 'distribution'})
freq_country.head()

,country_code,distribution
0,US,591612
1,CA,67970
2,GB,38380
3,AU,37103
4,DE,16007


In [25]:
#calcolo il numero totale di country_code non nulli nel dataframe lenders, utile per calcolare poi la percentuale
#della distribuzione precedentemente calcolata
num_tot_country_known = lenders['country_code'].notnull().sum()
num_tot_country_known

890539

In [26]:
#calcolo la percentuale che andrò poi ad utilizzare per sostituire i valori NaN di country_code presenti in lenders
freq_country['perc_distr'] = freq_country['distribution'] / num_tot_country_known * 100
freq_country.head()

,country_code,distribution,perc_distr
0,US,591612,66.433025
1,CA,67970,7.632456
2,GB,38380,4.309749
3,AU,37103,4.166353
4,DE,16007,1.797451


In [27]:
#utilizzo questo codice per far sì che la somma sia pari a 1
freq_country['perc_distr'] /= freq_country['perc_distr'].sum()

In [28]:
country_unknown = lenders.loc[lenders['country_code'].isnull()]

In [29]:
#imposto il seed su 1111 e vado a sostituire in modo randomico i valori nan dei country_code sconosciuti in base
#alle percentuali di distribuzione appena calcolati dei country conosciuti in modo da non avere più valori nan.
np.random.seed(1234)
country_unknown['country_code'] = np.random.choice(freq_country['country_code'], size=len(country_unknown.index), p = freq_country['perc_distr'])

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [30]:
country_unknown.head()

,lenders,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,US,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,US,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,US,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,AU,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,GB,1461301154,NaN,NaN,NaN,NaN,0


In [31]:
#vado ad aggiornare il file lenders con i dati che ho appena sistemato
lenders.update(country_unknown)

In [32]:
#ora unisco lenders_lent e lenders appena aggiornato in modo tale da avere il country_code collegato ad ogni lender
lent_country = pd.merge(lenders_lent, lenders[['lenders', 'country_code']], on = 'lenders', how = 'left')
lent_country = lent_country.rename(columns = {'loan_amount_per_lender': 'lent_amount'})

In [33]:
#è necessario ora fare la somma dei lent_amount raggruppando per country_code e ottengo così l'importo totale
#in uscita da tutti i country per finanziare dei prestiti.
lent_country_fin = lent_country.groupby(['country_code'])['lent_amount'].sum().reset_index()
lent_country_fin.head()

,country_code,lent_amount
0,AD,6.018512e+03
1,AE,1.954763e+06
2,AF,1.530532e+05
3,AG,1.229636e+03
4,AI,3.975203e+02


In [34]:
#eseguo lo stesso procedimento ma ora per ottenere la somma totale dei prestiti rispetto ai borrowed.
#raggruppo per country_name
borrowed_country = loans.groupby(['country_name'])['loan_amount'].sum().reset_index('country_name').rename(columns={'loan_amount': 'borrowed_amount'})

In [35]:
#collego la colonna country_code presente in country_stats alla borrowed_country
borrowed_country_fin = borrowed_country.merge(country_stats[['country_name', 'country_code']], on = 'country_name', how = 'left')
borrowed_country_fin.head()

,country_name,borrowed_amount,country_code
0,Afghanistan,1967950.0,AF
1,Albania,4307350.0,AL
2,Armenia,22950475.0,AM
3,Azerbaijan,14784625.0,AZ
4,Belize,150175.0,BZ


In [36]:
#avendo le due liste di lent e borrowed ora complete, posso unirle 
lent_and_borr = lent_country_fin.merge(borrowed_country_fin[['country_code', 'borrowed_amount']], on = 'country_code', how = 'left')

In [37]:
#ci sono dei country_code per i quali non risulta alcuna somma ricevuta (quindi rispetto ai borrowed), quindi
#imposto il valore pari a 0 con il codice fillna.
lent_and_borr['borrowed_amount'] = lent_and_borr['borrowed_amount'].fillna(0)

In [38]:
#aggiungo la colonna difference la quale mostri la differenza tra le due colonne lent and borrowed.
#I valori negativi nella colonna difference indicano un'entrata maggiore rispetto alle uscite.
lent_and_borr['difference'] = lent_and_borr['lent_amount'] - lent_and_borr['borrowed_amount']
lent_and_borr.head()

,country_code,lent_amount,borrowed_amount,difference
0,AD,6.018512e+03,0.0,6.018512e+03
1,AE,1.954763e+06,0.0,1.954763e+06
2,AF,1.530532e+05,1967950.0,-1.814897e+06
3,AG,1.229636e+03,0.0,1.229636e+03
4,AI,3.975203e+02,0.0,3.975203e+02


### 10. Which country has the highest ratio between the difference computed at the previous point and the population?

In [39]:
#le informazioni sulla popolazione si trovano nel csv country_stats quindi unisco tale colonna al precedente
#dataframe.
lent_and_borr = pd.merge(lent_and_borr, country_stats[['country_code', 'population']], on = 'country_code', how = 'left')
lent_and_borr.head()

,country_code,lent_amount,borrowed_amount,difference,population
0,AD,6.018512e+03,0.0,6.018512e+03,NaN
1,AE,1.954763e+06,0.0,1.954763e+06,9400145.0
2,AF,1.530532e+05,1967950.0,-1.814897e+06,35530081.0
3,AG,1.229636e+03,0.0,1.229636e+03,NaN
4,AI,3.975203e+02,0.0,3.975203e+02,NaN


In [40]:
#creo la colonna ratio dividendo i valori della differenza per il numero della popolazione di ogni country
lent_and_borr['ratio'] = lent_and_borr['difference'] / lent_and_borr['population']
lent_and_borr.head()

,country_code,lent_amount,borrowed_amount,difference,population,ratio
0,AD,6.018512e+03,0.0,6.018512e+03,NaN,NaN
1,AE,1.954763e+06,0.0,1.954763e+06,9400145.0,0.207950
2,AF,1.530532e+05,1967950.0,-1.814897e+06,35530081.0,-0.051081
3,AG,1.229636e+03,0.0,1.229636e+03,NaN,NaN
4,AI,3.975203e+02,0.0,3.975203e+02,NaN,NaN


In [41]:
#con questo codice trovo e mostro la riga in cui raggiungo il valore massimo di ratio
lent_and_borr[lent_and_borr['ratio']==lent_and_borr['ratio'].max()]

,country_code,lent_amount,borrowed_amount,difference,population,ratio
153,NO,2.070083e+07,0.0,2.070083e+07,5305383.0,3.901854


### 11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [42]:
#in questo caso mi serve trovare il numero della popolazione che non sta al di sotto della linea di povertà, quindi
#prima trovo il numero della popolazione che sta al di sotto della linea di povertà moltiplicando il tot della
#popolazione per la population_below_poverty_line (colonna che indica valori in percentuale) ed infine divido per
#100.
#dopo aver trovato il numero della popolazione al di sotto della linea di povertà, sottraggo quest'ultimo al tot
#della popolazione così trovando la restante popolazione che risulta invece al di sopra della linea di povertà.

country_stats['pop_below_poverty_not_perc'] = country_stats['population'] * country_stats['population_below_poverty_line'] / 100
country_stats['pop_notbelow_pov_line'] = country_stats['population'] - country_stats['pop_below_poverty_not_perc']
country_stats.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,pop_below_poverty_not_perc,pop_notbelow_pov_line
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,2.932804e+08,1.045900e+09
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,1.336204e+08,5.726589e+07
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,5.967343e+07,6.948984e+07
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan,5.811971e+07,1.388962e+08
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh,5.187097e+07,1.127988e+08


In [43]:
#unisco la colonna precedentemente creata al lent_and_borr
lent_and_borr = pd.merge(lent_and_borr, country_stats[['country_code', 'pop_notbelow_pov_line']], on = 'country_code', how = 'left')

In [44]:
#calcolo il rapporto tra la differenza tra lent and borrowed e la popolazione al di sopra della linea di povertà
#e mostro il risultato ottenuto
lent_and_borr['ratio2'] = lent_and_borr['difference'] / lent_and_borr['pop_notbelow_pov_line']
lent_and_borr.head()

,country_code,lent_amount,borrowed_amount,difference,population,ratio,pop_notbelow_pov_line,ratio2
0,AD,6.018512e+03,0.0,6.018512e+03,NaN,NaN,NaN,NaN
1,AE,1.954763e+06,0.0,1.954763e+06,9400145.0,0.207950,7.567117e+06,0.258323
2,AF,1.530532e+05,1967950.0,-1.814897e+06,35530081.0,-0.051081,2.281031e+07,-0.079565
3,AG,1.229636e+03,0.0,1.229636e+03,NaN,NaN,NaN,NaN
4,AI,3.975203e+02,0.0,3.975203e+02,NaN,NaN,NaN,NaN


In [45]:
#con questo codice trovo e mostro la riga in cui raggiungo il valore massimo di ratio
lent_and_borr[lent_and_borr['ratio2']==lent_and_borr['ratio2'].max()]

,country_code,lent_amount,borrowed_amount,difference,population,ratio,pop_notbelow_pov_line,ratio2
35,CA,9.475684e+07,50000.0,9.470684e+07,36624199.0,2.585909,3.318152e+07,2.854204


### 12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year.
### For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [46]:
#essendo già a conoscenza della presenza di valori NAN sia nella colonna disburse_time che nella colonna 
#planned_expiration_time, creo una lista in cui sono presenti solo i loan non nulli (così da non modificare il 
#dataframe iniziale) ed elimino l'orario così da avere un calcolo pulito. 
loans_time = date_loans[['loan_id', 'disburse_time', 'planned_expiration_time', 'loan_amount']]

In [47]:
loans_time['disburse_time'] = loans_time['disburse_time'].dt.tz_localize(None)
loans_time['planned_expiration_time'] = loans_time['planned_expiration_time'].dt.tz_localize(None)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
loans_time['disburse_time'] = loans_time['disburse_time'].dt.normalize()
loans_time['planned_expiration_time'] = loans_time['planned_expiration_time'].dt.normalize()

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [49]:
loans_time.head()

,loan_id,disburse_time,planned_expiration_time,loan_amount
0,657307,2013-12-22,2014-02-14,125.0
1,657259,2013-12-20,2014-03-26,400.0
2,658010,2014-01-09,2014-02-15,400.0
3,659347,2014-01-17,2014-02-21,625.0
4,656933,2013-12-17,2014-02-13,425.0


In [50]:
#creo le due diverse liste. una in cui l'anno di esborso e scadenza del loan_id coincide, l'altra in cui invece i
#due differiscono.
loans_same_year = loans_time.loc[loans_time['disburse_time'].dt.year == loans_time['planned_expiration_time'].dt.year]
loans_different_year = loans_time[loans_time['disburse_time'].dt.year != loans_time['planned_expiration_time'].dt.year]

In [51]:
#calcolo il numero di giorni che intercorrono tra esborso e scadenza (stesso anno)
loans_same_year['number_days'] = (loans_same_year['planned_expiration_time'] - loans_same_year['disburse_time']).dt.days +1

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [52]:
#considerata la presenza di anni bisestili quali 2012 e 2016, imposto l'aggiunta non di 1 ma di 2 giorni ai
#prestiti per i quali il disburse time parta dal mese di gennaio o dal mese di febbraio di uno di questi due anni
#bisestili.

In [53]:
loans_same_year.loc[(loans_same_year['disburse_time'].dt.year == 2012 ) & (loans_same_year['disburse_time'].dt.month == 1), 'number_days'] = (loans_same_year['planned_expiration_time'] - loans_same_year['disburse_time']).dt.days +2

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [54]:
loans_same_year.loc[(loans_same_year['disburse_time'].dt.year == 2012 ) & (loans_same_year['disburse_time'].dt.month == 2), 'number_days'] = (loans_same_year['planned_expiration_time'] - loans_same_year['disburse_time']).dt.days +2

In [55]:
loans_same_year.loc[(loans_same_year['disburse_time'].dt.year == 2016 ) & (loans_same_year['disburse_time'].dt.month == 1), 'number_days'] = (loans_same_year['planned_expiration_time'] - loans_same_year['disburse_time']).dt.days +2

In [56]:
loans_same_year.loc[(loans_same_year['disburse_time'].dt.year == 2016 ) & (loans_same_year['disburse_time'].dt.month == 2), 'number_days'] = (loans_same_year['planned_expiration_time'] - loans_same_year['disburse_time']).dt.days +2

In [57]:
loans_same_year.head()

,loan_id,disburse_time,planned_expiration_time,loan_amount,number_days
2,658010,2014-01-09,2014-02-15,400.0,38
3,659347,2014-01-17,2014-02-21,625.0,36
5,659605,2014-01-15,2014-02-20,350.0,37
6,660240,2014-01-20,2014-02-21,125.0,33
7,661601,2014-01-10,2014-02-25,1600.0,47


In [58]:
#nel caso dei loans_different_year non mi basta sapere quanti giorni intercorrono dal disburse_time al planned_
#_expiration_time, bensì quanti giorni sono attribuiti ad uno così sapendo che i restanti giorni sono attribuiti
#all'altro. può essere utile quindi inserire l'inizio e la fine dell'anno così da fare i dovuti calcoli

In [59]:
#utilizzo questo codice per avere il numero di righe di ogni loan_id pari al numero di anni in cui il prestito
#è attivo (quindi se l'esborso avviene nel 2014 e il prestito scade nel 2015 avrò due righe per tale loan_id, se
#scadesse nel 2016 invece avrei tre righe per tale loan_id)
loans_different_year = loans_different_year.loc[loans_different_year.index.repeat(loans_different_year.planned_expiration_time.dt.year - loans_different_year.disburse_time.dt.year + 1)]

In [60]:
#vado ora a creare la colonna beginning_year la quale nella prima riga di un dato loan_id presenti il primo giorno
#solare dell'anno in cui avviene l'esborso (es: 2013/1/1)
loans_different_year['beginning_year'] = loans_different_year.drop_duplicates()['disburse_time'].dt.year.apply(lambda x: datetime.datetime(x, 1, 1))

In [61]:
y = loans_different_year['disburse_time'].dt.year
loans_different_year['beginning_year'] = pd.to_datetime(loans_different_year.groupby(loans_different_year['loan_id']).cumcount() + y, format='%Y')

In [62]:
#svolgo lo stesso procedimento utilizzato per il primo giorno dell'anno, per la fine dell'anno quindi otterrò
#per ogni riga di un dato loan_id l'anno corrispondente (che sia esborso, n+1, scadenza) con l'ultimo giorno
#solare (es: 2013/12/31)
loans_different_year['end_year'] = pd.to_datetime(loans_different_year.groupby(loans_different_year['loan_id']).cumcount() + y, format='%Y')

In [63]:
loans_different_year['end_year'] = loans_different_year['beginning_year'].dt.year.apply(lambda x: datetime.datetime(x, 12, 31))

In [64]:
#resetto l'indice del dataframe
loans_different_year = loans_different_year.reset_index()

In [65]:
#imposto il calcolo logico per la distribuzione di ogni prestito nei differenti anni in cui ha durata
loans_different_year.loc[(loans_different_year['planned_expiration_time'].dt.year > loans_different_year['end_year'].dt.year) & (loans_different_year['disburse_time'].dt.year == loans_different_year['beginning_year'].dt.year), 'days_per_year'] = (loans_different_year['end_year'] - loans_different_year['disburse_time']).dt.days +1

In [66]:
loans_different_year.loc[(loans_different_year['planned_expiration_time'].dt.year > loans_different_year['end_year'].dt.year) & (loans_different_year['disburse_time'].dt.year != loans_different_year['beginning_year'].dt.year), 'days_per_year'] = (loans_different_year['end_year'] - loans_different_year['beginning_year']).dt.days +1

In [67]:
loans_different_year.loc[(loans_different_year['planned_expiration_time'].dt.year == loans_different_year['end_year'].dt.year), 'days_per_year'] = (loans_different_year['planned_expiration_time'] - loans_different_year['beginning_year']).dt.days +1

In [68]:
#creo la colonna in cui mostro il totale giorni del prestito (mostrerà il totale in ogni riga del dato loan_id)
loans_different_year['tot_loan_days'] = (loans_different_year['planned_expiration_time'] - loans_different_year['disburse_time']).dt.days +1
loans_different_year.head()

,index,loan_id,disburse_time,planned_expiration_time,loan_amount,beginning_year,end_year,days_per_year,tot_loan_days
0,0,657307,2013-12-22,2014-02-14,125.0,2013-01-01,2013-12-31,10.0,55
1,0,657307,2013-12-22,2014-02-14,125.0,2014-01-01,2014-12-31,45.0,55
2,1,657259,2013-12-20,2014-03-26,400.0,2013-01-01,2013-12-31,12.0,97
3,1,657259,2013-12-20,2014-03-26,400.0,2014-01-01,2014-12-31,85.0,97
4,4,656933,2013-12-17,2014-02-13,425.0,2013-01-01,2013-12-31,15.0,59


In [69]:
#applico ora l'equazione esempio nella consegna del punto in questione.
loans_different_year['loan_amount_per_year'] = (loans_different_year['loan_amount']* loans_different_year['days_per_year'])/loans_different_year['tot_loan_days']
loans_different_year.head()

,index,loan_id,disburse_time,planned_expiration_time,loan_amount,beginning_year,end_year,days_per_year,tot_loan_days,loan_amount_per_year
0,0,657307,2013-12-22,2014-02-14,125.0,2013-01-01,2013-12-31,10.0,55,22.727273
1,0,657307,2013-12-22,2014-02-14,125.0,2014-01-01,2014-12-31,45.0,55,102.272727
2,1,657259,2013-12-20,2014-03-26,400.0,2013-01-01,2013-12-31,12.0,97,49.484536
3,1,657259,2013-12-20,2014-03-26,400.0,2014-01-01,2014-12-31,85.0,97,350.515464
4,4,656933,2013-12-17,2014-02-13,425.0,2013-01-01,2013-12-31,15.0,59,108.050847


In [70]:
#calcolo la somma del loan raggruppando per beginning_year
loans_different_year_total = loans_different_year.groupby(loans_different_year['beginning_year'].dt.year)['loan_amount_per_year'].sum().to_frame().reset_index()
loans_different_year_total.head(10)

,beginning_year,loan_amount_per_year
0,2011,6.614928e+05
1,2012,9.251024e+06
2,2013,2.512557e+07
3,2014,3.001850e+07
4,2015,2.394231e+07
5,2016,2.081552e+07
6,2017,2.046922e+07
7,2018,4.877086e+06


In [71]:
#faccio lo stesso calcolo ma raggruppando per disburse time
loans_same_year_tot = loans_same_year.groupby(loans_same_year['disburse_time'].dt.year)['loan_amount'].sum().to_frame().reset_index()
loans_same_year_tot.head(10)

,disburse_time,loan_amount
0,2012,104138000.0
1,2013,99878775.0
2,2014,123149050.0
3,2015,135779450.0
4,2016,138206675.0
5,2017,151284475.0
6,2018,710425.0


In [72]:
#unisco i dataframe same e different in modo da avere tutti i valori in un unico dataframe
tot_loan_per_year = pd.merge(loans_different_year_total, loans_same_year_tot, left_on = 'beginning_year', right_on = 'disburse_time', how = 'outer')
tot_loan_per_year.head(10)

,beginning_year,loan_amount_per_year,disburse_time,loan_amount
0,2011,6.614928e+05,NaN,NaN
1,2012,9.251024e+06,2012.0,104138000.0
2,2013,2.512557e+07,2013.0,99878775.0
3,2014,3.001850e+07,2014.0,123149050.0
4,2015,2.394231e+07,2015.0,135779450.0
5,2016,2.081552e+07,2016.0,138206675.0
6,2017,2.046922e+07,2017.0,151284475.0
7,2018,4.877086e+06,2018.0,710425.0


In [73]:
#non risulta l'anno 2011 nel caso del dataframe loans_different_year total quindi correggo questo missing value
#e imposto l'importo del loan di tale anno pari a 0
tot_loan_per_year[['loan_amount_per_year', 'loan_amount']] = tot_loan_per_year[['loan_amount_per_year', 'loan_amount']].fillna(0)
tot_loan_per_year['beginning_year'] = tot_loan_per_year['beginning_year'].fillna(tot_loan_per_year['disburse_time'])
tot_loan_per_year['disburse_time'] = tot_loan_per_year['disburse_time'].fillna(tot_loan_per_year['beginning_year'])

In [74]:
#infine, sommo le due colonne dei due diversi totali così ottenendo una somma finale che rappresenti l'intero
#ammontare di loan per ogni anno.
tot_loan_per_year['tot_per_year'] = tot_loan_per_year['loan_amount_per_year'] + tot_loan_per_year['loan_amount']
tot_loan_per_year = tot_loan_per_year[['beginning_year', 'tot_per_year']]
tot_loan_per_year.head(10)

,beginning_year,tot_per_year
0,2011,6.614928e+05
1,2012,1.133890e+08
2,2013,1.250043e+08
3,2014,1.531675e+08
4,2015,1.597218e+08
5,2016,1.590222e+08
6,2017,1.717537e+08
7,2018,5.587511e+06
